In [18]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup as bs
from datetime import datetime
import time
import random
import warnings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 10)

In [19]:
# Load the DataFrames
df = pd.read_csv('exports/stocks.csv')
df

,stock_name,stock_code
0,7-ELEVEN MALAYSIA HOLDINGS BERHAD,5250
1,ABF MALAYSIA BOND INDEX FUND,0800EA
2,ABLE GLOBAL BERHAD,7167
3,ABLEGROUP BERHAD,7086
4,ABM FUJIYA BERHAD,5198
...,...,...
1008,YKGI HOLDINGS BERHAD,7020
1009,YLI HOLDINGS BERHAD,7014
1010,YNH PROPERTY BERHAD,3158
1011,YONG TAI BERHAD,7066


In [20]:
# Put it into a list called stock_code
stock_code = list(df['stock_code'])
stock_code

['5250',
 '0800EA',
 '7167',
 '7086',
 '5198',
 '03028',
 '7131',
 '0218',
 '0122',
 '1481',
 '5281',
 '9148',
 '7191',
 '7146',
 '0181',
 '6599',
 '5139',
 '5185',
 '7145',
 '0258',
 '7315',
 '7078',
 '0209',
 '5238',
 '2658',
 '7609',
 '5116',
 '5115',
 '2674',
 '0079',
 '2488',
 '1163',
 '03051',
 '5269',
 '5127',
 '5293',
 '5307',
 '5120',
 '03011',
 '1015',
 '7031',
 '6351',
 '7083',
 '0048',
 '4758',
 '0226',
 '6556',
 '5082',
 '5568',
 '5088',
 '7090',
 '5015',
 '6432',
 '0119',
 '7214',
 '7181',
 '7007',
 '0038',
 '0068',
 '7722',
 '7129',
 '0159',
 '0105',
 '4057',
 '7162',
 '7054',
 '03032',
 '6399',
 '0072',
 '8176',
 '7048',
 '5130',
 '7099',
 '5302',
 '03037',
 '8885',
 '5204',
 '7579',
 '6888',
 '5106',
 '7120',
 '2305',
 '5021',
 '7005',
 '03012',
 '0098',
 '5258',
 '7251',
 '1899',
 '6602',
 '0187',
 '5190',
 '3239',
 '3395',
 '5196',
 '4219',
 '5248',
 '9814',
 '7668',
 '0263',
 '6173',
 '5932',
 '0195',
 '6998',
 '5032',
 '0179',
 '5069',
 '0168',
 '9288',
 '7036',
 '

In [25]:
# Create an empty dataframe
div_info_final = pd.DataFrame()

# Connect/open to Chrome webdriver 
driver = webdriver.Chrome()
driver.implicitly_wait(60)

timeout_list = []
no_div=[]

for s in stock_code:
    status = ""
    try:
        url = 'https://www.klsescreener.com/v2/stocks/view/' + s
        driver.get(url) 
        content = driver.page_source
        soup = bs(content)

        # Scrap table
        tables = soup.find_all('table')
        # Read tables with Pandas read_html()
        dfs = pd.read_html(str(tables))
        
        # Find the table that contain column 'Payment Date'
        i = 0
        has_div = False
        while i < len(dfs) and not has_div:
            n = 0
            while n < len(dfs[i].columns):
                if dfs[i].columns[n] == 'Payment Date':
                    has_div = True
                n += 1
            i += 1

        if has_div:
            div_info2 = dfs[i - 1]
           

            # Change data types    
            div_info2['EX Date'] = pd.to_datetime(div_info2['EX Date'])
            div_info2['Financial Year'] = pd.to_datetime(div_info2['Financial Year'])
            #print(div_info2['EX Date'])
            div_info2['EX Date'] = pd.to_datetime(div_info2['EX Date'].dt.strftime('%Y-%m-%d'))


            # Filter data between two dates   
            filtered_df = div_info2.loc[(div_info2['EX Date'] >= '2018-01-01')
                                                & (div_info2['EX Date'] < datetime.now())]
            filtered_df.reset_index(drop=True, inplace=True)
            
            # Check whether the filtered_df is empty
            if filtered_df.empty:
                status = 'N'
                no_div.append(s)
                print('Stock code: ' + s + ' no dividend info between 2018 to 2022')

            else:
                status = 'Y'
                filtered_df['Stock Code'] = s
                filtered_df['Year'] = filtered_df['Financial Year'].dt.strftime("%Y")
                filtered_df['Status'] = status
                div_info_final = div_info_final.append(filtered_df, ignore_index=True)
                print('Stock code: ' + s + ' yes')
        else:
            status = 'N'
            no_div.append(s)
            print('Stock code: ' + s + ' no dividend info')

        
    except Exception as e:
        print("Stock code :" + s + " Loading failed due to Timeout")
        timeout_list.append(s)
        print(e)

    # Program to generate a random number between 5 to 30
    # Importing the random module
    sec = random.randint(3, 8)

    # Program scrapping waiting time
    time.sleep(4)  # seconds  

C:\Users\Aidan\AppData\Local\Temp\ipykernel_6036\2913752897.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Stock Code'] = s
C:\Users\Aidan\AppData\Local\Temp\ipykernel_6036\2913752897.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Year'] = filtered_df['Financial Year'].dt.strftime("%Y")
C:\Users\Aidan\AppData\Local\Temp\ipykernel_6036\2913752897.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

Stock code: 5250 yes


C:\Users\Aidan\AppData\Local\Temp\ipykernel_6036\2913752897.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Stock Code'] = s
C:\Users\Aidan\AppData\Local\Temp\ipykernel_6036\2913752897.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Year'] = filtered_df['Financial Year'].dt.strftime("%Y")
C:\Users\Aidan\AppData\Local\Temp\ipykernel_6036\2913752897.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

Stock code: 0800EA yes


C:\Users\Aidan\AppData\Local\Temp\ipykernel_6036\2913752897.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Stock Code'] = s
C:\Users\Aidan\AppData\Local\Temp\ipykernel_6036\2913752897.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Year'] = filtered_df['Financial Year'].dt.strftime("%Y")
C:\Users\Aidan\AppData\Local\Temp\ipykernel_6036\2913752897.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

...

Stock code :5142 Loading failed due to Timeout
Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=119.0.6045.105)
Stacktrace:
	GetHandleVerifier [0x00007FF60A3E82B2+55298]
	(No symbol) [0x00007FF60A355E02]
	(No symbol) [0x00007FF60A2105AB]
	(No symbol) [0x00007FF60A1F0038]
	(No symbol) [0x00007FF60A276BC7]
	(No symbol) [0x00007FF60A28A15F]
	(No symbol) [0x00007FF60A271E83]
	(No symbol) [0x00007FF60A24670A]
	(No symbol) [0x00007FF60A247964]
	GetHandleVerifier [0x00007FF60A760AAB+3694587]
	GetHandleVerifier [0x00007FF60A7B728E+4048862]
	GetHandleVerifier [0x00007FF60A7AF173+4015811]
	GetHandleVerifier [0x00007FF60A4847D6+695590]
	(No symbol) [0x00007FF60A360CE8]
	(No symbol) [0x00007FF60A35CF34]
	(No symbol) [0x00007FF60A35D062]
	(No symbol) [0x00007FF60A34D3A3]
	BaseThreadInitThunk [0x00007FFF76547344+20]
	RtlUserThreadStart [0x00007FFF769C26B1+33]

Stock code :0102 Loading failed due to Timeout
Message: no such window: t

Stock code :0271 Loading failed due to Timeout
Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=119.0.6045.105)
Stacktrace:
	GetHandleVerifier [0x00007FF60A3E82B2+55298]
	(No symbol) [0x00007FF60A355E02]
	(No symbol) [0x00007FF60A2105AB]
	(No symbol) [0x00007FF60A1F0038]
	(No symbol) [0x00007FF60A276BC7]
	(No symbol) [0x00007FF60A28A15F]
	(No symbol) [0x00007FF60A271E83]
	(No symbol) [0x00007FF60A24670A]
	(No symbol) [0x00007FF60A247964]
	GetHandleVerifier [0x00007FF60A760AAB+3694587]
	GetHandleVerifier [0x00007FF60A7B728E+4048862]
	GetHandleVerifier [0x00007FF60A7AF173+4015811]
	GetHandleVerifier [0x00007FF60A4847D6+695590]
	(No symbol) [0x00007FF60A360CE8]
	(No symbol) [0x00007FF60A35CF34]
	(No symbol) [0x00007FF60A35D062]
	(No symbol) [0x00007FF60A34D3A3]
	BaseThreadInitThunk [0x00007FFF76547344+20]
	RtlUserThreadStart [0x00007FFF769C26B1+33]

Stock code :5246 Loading failed due to Timeout
Message: no such window: t

Stock code :5156 Loading failed due to Timeout
Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=119.0.6045.105)
Stacktrace:
	GetHandleVerifier [0x00007FF60A3E82B2+55298]
	(No symbol) [0x00007FF60A355E02]
	(No symbol) [0x00007FF60A2105AB]
	(No symbol) [0x00007FF60A1F0038]
	(No symbol) [0x00007FF60A276BC7]
	(No symbol) [0x00007FF60A28A15F]
	(No symbol) [0x00007FF60A271E83]
	(No symbol) [0x00007FF60A24670A]
	(No symbol) [0x00007FF60A247964]
	GetHandleVerifier [0x00007FF60A760AAB+3694587]
	GetHandleVerifier [0x00007FF60A7B728E+4048862]
	GetHandleVerifier [0x00007FF60A7AF173+4015811]
	GetHandleVerifier [0x00007FF60A4847D6+695590]
	(No symbol) [0x00007FF60A360CE8]
	(No symbol) [0x00007FF60A35CF34]
	(No symbol) [0x00007FF60A35D062]
	(No symbol) [0x00007FF60A34D3A3]
	BaseThreadInitThunk [0x00007FFF76547344+20]
	RtlUserThreadStart [0x00007FFF769C26B1+33]

Stock code :5267 Loading failed due to Timeout
Message: no such window: t

Stock code :0025 Loading failed due to Timeout
Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=119.0.6045.105)
Stacktrace:
	GetHandleVerifier [0x00007FF60A3E82B2+55298]
	(No symbol) [0x00007FF60A355E02]
	(No symbol) [0x00007FF60A2105AB]
	(No symbol) [0x00007FF60A1F0038]
	(No symbol) [0x00007FF60A276BC7]
	(No symbol) [0x00007FF60A28A15F]
	(No symbol) [0x00007FF60A271E83]
	(No symbol) [0x00007FF60A24670A]
	(No symbol) [0x00007FF60A247964]
	GetHandleVerifier [0x00007FF60A760AAB+3694587]
	GetHandleVerifier [0x00007FF60A7B728E+4048862]
	GetHandleVerifier [0x00007FF60A7AF173+4015811]
	GetHandleVerifier [0x00007FF60A4847D6+695590]
	(No symbol) [0x00007FF60A360CE8]
	(No symbol) [0x00007FF60A35CF34]
	(No symbol) [0x00007FF60A35D062]
	(No symbol) [0x00007FF60A34D3A3]
	BaseThreadInitThunk [0x00007FFF76547344+20]
	RtlUserThreadStart [0x00007FFF769C26B1+33]

Stock code :5300 Loading failed due to Timeout
Message: no such window: t

Stock code :5159 Loading failed due to Timeout
Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=119.0.6045.105)
Stacktrace:
	GetHandleVerifier [0x00007FF60A3E82B2+55298]
	(No symbol) [0x00007FF60A355E02]
	(No symbol) [0x00007FF60A2105AB]
	(No symbol) [0x00007FF60A1F0038]
	(No symbol) [0x00007FF60A276BC7]
	(No symbol) [0x00007FF60A28A15F]
	(No symbol) [0x00007FF60A271E83]
	(No symbol) [0x00007FF60A24670A]
	(No symbol) [0x00007FF60A247964]
	GetHandleVerifier [0x00007FF60A760AAB+3694587]
	GetHandleVerifier [0x00007FF60A7B728E+4048862]
	GetHandleVerifier [0x00007FF60A7AF173+4015811]
	GetHandleVerifier [0x00007FF60A4847D6+695590]
	(No symbol) [0x00007FF60A360CE8]
	(No symbol) [0x00007FF60A35CF34]
	(No symbol) [0x00007FF60A35D062]
	(No symbol) [0x00007FF60A34D3A3]
	BaseThreadInitThunk [0x00007FFF76547344+20]
	RtlUserThreadStart [0x00007FFF769C26B1+33]



In [5]:
div_info_final

,Announced,Financial Year,Subject,EX Date,Payment Date,Amount,Indicator,Unnamed: 7,Stock Code,Year,Status
0,13 Apr 2023,2022-12-31,Interim Dividend,10 May 2023,25 May 2023,0.054,Currency,View,5250,2022,Y
1,20 Apr 2022,2021-12-31,Interim Dividend,10 May 2022,27 May 2022,0.026,Currency,View,5250,2021,Y
2,15 Apr 2021,2020-12-31,Interim Dividend,05 May 2021,21 May 2021,0.016,Currency,View,5250,2020,Y
3,22 Apr 2020,2019-12-31,Interim Dividend,12 May 2020,28 May 2020,0.023,Currency,View,5250,2019,Y
4,29 Apr 2019,2018-12-31,Interim Dividend,09 May 2019,23 May 2019,0.024,Currency,View,5250,2018,Y
...,...,...,...,...,...,...,...,...,...,...,...
4506,25 Jun 2020,2020-06-30,Interim Dividend,28 Jul 2020,18 Aug 2020,0.015,Currency,View,5159,2020,Y
4507,21 Oct 2019,2019-06-30,Final Dividend,19 Dec 2019,17 Jan 2020,0.025,Currency,View,5159,2019,Y
4508,30 May 2019,2019-06-30,Interim Dividend,08 Jul 2019,26 Jul 2019,0.025,Currency,View,5159,2019,Y
4509,30 Oct 2018,2018-06-30,Final Dividend,19 Dec 2018,18 Jan 2019,0.020,Currency,View,5159,2018,Y


In [6]:
div_info_final = div_info_final.drop(['Unnamed: 7'], axis=1)
div_info_final

,Announced,Financial Year,Subject,EX Date,Payment Date,Amount,Indicator,Stock Code,Year,Status
0,13 Apr 2023,2022-12-31,Interim Dividend,10 May 2023,25 May 2023,0.054,Currency,5250,2022,Y
1,20 Apr 2022,2021-12-31,Interim Dividend,10 May 2022,27 May 2022,0.026,Currency,5250,2021,Y
2,15 Apr 2021,2020-12-31,Interim Dividend,05 May 2021,21 May 2021,0.016,Currency,5250,2020,Y
3,22 Apr 2020,2019-12-31,Interim Dividend,12 May 2020,28 May 2020,0.023,Currency,5250,2019,Y
4,29 Apr 2019,2018-12-31,Interim Dividend,09 May 2019,23 May 2019,0.024,Currency,5250,2018,Y
...,...,...,...,...,...,...,...,...,...,...
4506,25 Jun 2020,2020-06-30,Interim Dividend,28 Jul 2020,18 Aug 2020,0.015,Currency,5159,2020,Y
4507,21 Oct 2019,2019-06-30,Final Dividend,19 Dec 2019,17 Jan 2020,0.025,Currency,5159,2019,Y
4508,30 May 2019,2019-06-30,Interim Dividend,08 Jul 2019,26 Jul 2019,0.025,Currency,5159,2019,Y
4509,30 Oct 2018,2018-06-30,Final Dividend,19 Dec 2018,18 Jan 2019,0.020,Currency,5159,2018,Y


In [7]:
# Specify the file path where you want to save the CSV file
csv_file_path = 'exports/historicaldividends.csv'

# Use the to_csv() method to export the DataFrame to a CSV file
div_info_final.to_csv(csv_file_path, index=False)  # Set index=False to exclude the DataFrame index from the CSV

print(f"DataFrame saved to {csv_file_path}")

DataFrame saved to exports/historicaldividends.csv


In [8]:
df = pd.read_csv('exports/historicaldividends.csv')


In [9]:
# Assuming your DataFrame is named 'df'
columns_to_remove = ["Financial Year", "Subject", "Indicator", "Year", "Status"]
df = df.drop(columns=columns_to_remove)


In [10]:
# Convert 'Announced' column to datetime
df['Announced'] = pd.to_datetime(df['Announced'])
# Convert 'EX Date' and 'Payment Date' columns to datetime
df['EX Date'] = pd.to_datetime(df['EX Date'])
df['Payment Date'] = pd.to_datetime(df['Payment Date'])
# Convert 'Amount' column to numeric (float)
df['Amount'] = pd.to_numeric(df['Amount'], errors='coerce')  # 'coerce' will set non-numeric values to NaN

In [11]:
print(df.dtypes)

Announced       datetime64[ns]
EX Date         datetime64[ns]
Payment Date    datetime64[ns]
Amount                 float64
Stock Code              object
dtype: object


In [12]:
df = df.rename(columns={
    "Announced": "dAnnceDate",
    "EX Date": "dExDate",
    "Payment Date": "dPayDate",
    "Amount": "dAmount",
    "Stock Code": "stock_code"
})
df

,dAnnceDate,dExDate,dPayDate,dAmount,stock_code
0,2023-04-13,2023-05-10,2023-05-25,0.054,5250
1,2022-04-20,2022-05-10,2022-05-27,0.026,5250
2,2021-04-15,2021-05-05,2021-05-21,0.016,5250
3,2020-04-22,2020-05-12,2020-05-28,0.023,5250
4,2019-04-29,2019-05-09,2019-05-23,0.024,5250
...,...,...,...,...,...
4506,2020-06-25,2020-07-28,2020-08-18,0.015,5159
4507,2019-10-21,2019-12-19,2020-01-17,0.025,5159
4508,2019-05-30,2019-07-08,2019-07-26,0.025,5159
4509,2018-10-30,2018-12-19,2019-01-18,0.020,5159


In [13]:
# Specify the file path where you want to save the CSV file
csv_file_path = 'exports/historicaldividends_cleaned.csv'

# Use the to_csv() method to export the DataFrame to a CSV file
df.to_csv(csv_file_path, index=False)  # Set index=False to exclude the DataFrame index from the CSV

print(f"DataFrame saved to {csv_file_path}")

DataFrame saved to exports/historicaldividends_cleaned.csv
